In [8]:
! pip install rembg

In [9]:
! pip install backgroundremover


In [1]:
! pip install transformers



In [2]:
! pip install --upgrade --force-reinstall pillow


  Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
backgroundremover 0.3.0 requires Pillow<10.0.0,>=8.1.1, but you have pillow 11.1.0 which is incompatible.


In [3]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [5]:
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_image_features(image_paths):
    images = [Image.open(path).convert("RGB") for path in image_paths]
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    return image_features

mcafee_logos = get_image_features(["/content/mcafee-removebg-preview.png", "/content/mcafee_ver-removebg-preview.png"])
mcafee_shield_logos = get_image_features(["/content/Screenshot_2025-04-05_212545-removebg-preview.png"])


In [31]:
from PIL import Image

# Load and process the image
user_image = Image.open("/content/user_input_1_nobg.png").convert("RGB")
inputs = processor(images=user_image, return_tensors="pt").to(device)

# Encode using the model
with torch.no_grad():
    user_feature = model.get_image_features(**inputs)
    user_feature = user_feature / user_feature.norm(dim=-1, keepdim=True)


In [32]:
def average_cosine_similarity(user_feat, logo_feats):
    sims = torch.nn.functional.cosine_similarity(user_feat, logo_feats.squeeze(1), dim=-1)
    return sims.mean().item()

scores = {
    "McAfee": average_cosine_similarity(user_feature, mcafee_logos),
    "Shield": average_cosine_similarity(user_feature, mcafee_shield_logos),

}

for company, sim in scores.items():
    print(f"{company} similarity: {sim:.4f}")

predicted_class = max(scores, key=scores.get)
print("Predicted class:", predicted_class)

#Got Correct Output

McAfee similarity: 0.8420
Shield similarity: 0.7631
Predicted class: McAfee


In [29]:
import cv2
import numpy as np

def remove_background(image_path, output_path="output_no_bg.png"):
    # Load image
    image = cv2.imread(image_path)
    mask = np.zeros(image.shape[:2], np.uint8)

    # Background and foreground models (used internally by grabCut)
    bgModel = np.zeros((1, 65), np.float64)
    fgModel = np.zeros((1, 65), np.float64)

    # Define a rectangle around the subject (you can tweak this)
    height, width = image.shape[:2]
    rect = (10, 10, width - 20, height - 20)

    # Apply grabCut
    cv2.grabCut(image, mask, rect, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)

    # Convert mask to binary: 0 or 1
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Remove background
    result = image * mask2[:, :, np.newaxis]

    # Convert to BGRA (add alpha channel)
    result_with_alpha = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result_with_alpha[:, :, 3] = mask2 * 255  # Set alpha channel

    # Save result
    cv2.imwrite(output_path, result_with_alpha)

    return output_path


In [30]:
remove_background("/content/user_input_1.png", "/content/user_input_1_nobg.png")


'/content/user_input_1_nobg.png'